# Book Recommender

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings


from keras.layers import Input, Embedding, Flatten, Dot, Dense, Reshape, Concatenate, Dropout
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras import regularizers

from sklearn.model_selection import train_test_split

#warnings.filterwarnings('ignore')
%matplotlib inline
# TIMESTAMP IMPORT MUST BE CHANGED WHEN USING PANDAS 23.0+
#from pandas.lib import Timestamp
# CHANGE TO THE FOLLOWING
from pandas._libs.tslibs.timestamps import Timestamp

%matplotlib inline is not supported in Databricks.
You can display matplotlib figures using display(). For an example, see https://docs.databricks.com/user-guide/visualizations/matplotlib-and-ggplot.html

##Import book rating data

In [4]:
#use mount point you previously created when setting up the cluster
# Open and read the file from mounted storage
container = "data/5.Recommenders"
inputFilePath = "/dbfs/mnt/{}/{}".format(container, 'ratings.csv') 
dataset = pd.read_csv(inputFilePath)
dataset.head()

Out[2]: 
 book_id user_id rating
0 1 314 5
1 1 439 3
2 1 588 5
3 1 1169 4
4 1 1185 4

Let's have a look at how many data points we have

In [6]:
dataset.shape

Out[6]: (981756, 3)

Let's check how many users and items we have

In [8]:
n_users = len(dataset.user_id.unique())
print("Number of users: ", n_users)
n_items = len(dataset.book_id.unique())
print("Number of items: ", n_items)

Number of users: 53424
Number of items: 10000

To visualize the model, you can use Netron https://lutzroeder.github.io/netron/ in a browser.
Download the h5 file locally and point Netron to it.

Read book data

In [11]:
inputFilePath = "/dbfs/mnt/{}/{}".format(container, 'books.csv') 
books = pd.read_csv(inputFilePath)
books.head(10)

Out[38]: 
 ...
0 ...
1 ...
2 ...
3 ...
4 ...
5 ...
6 ...
7 ...
8 ...
9 ...

[10 rows x 23 columns]

In [12]:
books.loc[0]

Out[47]: 
id 1
book_id 2767052
best_book_id 2767052
work_id 2792775
books_count 272
isbn 439023483
isbn13 9.78044e+12
authors Suzanne Collins
original_publication_year 2008
original_title The Hunger Games
title The Hunger Games (The Hunger Games, #1)
language_code eng
average_rating 4.34
ratings_count 4780653
work_ratings_count 4942365
work_text_reviews_count 155254
ratings_1 66715
ratings_2 127936
ratings_3 560092
ratings_4 1481305
ratings_5 2706317
image_url https://images.gr-assets.com/books/1447303603m...
small_image_url https://images.gr-assets.com/books/1447303603s...
Name: 0, dtype: object

## Add item metadata
We extend the model with data pertaining to the items. This requires numerical embedding of titles

In [14]:
books_subset=books[['id', 'original_title']]
books_subset=books_subset.rename({'id':'book_id'}, axis='columns')

books_subset.head(10)

Out[34]: 
 ...
0 ...
1 ...
2 ...
3 ...
4 ...
5 ...
6 ...
7 ...
8 ...
9 ...

[10 rows x 2 columns]

In [15]:
#create a new dataset by joining the title information
new_dataset=dataset.join(books_subset.set_index('book_id'), on='book_id')
new_dataset.head()
n_metadata=len(new_dataset.original_title.unique())
n_metadata

Out[5]: 9275

Embedding only works on integer vectors. We need to encode the titles.

In [17]:
from keras.preprocessing.text import Tokenizer

titles=np.array(new_dataset.original_title)

t=Tokenizer()
titles=[str(i) for i in new_dataset.original_title]
t.fit_on_texts(titles)

vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_titles = t.texts_to_sequences(titles)

from keras.preprocessing.sequence import pad_sequences
max_length = 10
padded_titles = pad_sequences(encoded_titles, 
                              maxlen=max_length, 
                              padding='post')
padded_titles

Out[6]: 
array([[ 1, 965, 525, ..., 0, 0, 0],
 [ 1, 965, 525, ..., 0, 0, 0],
 [ 1, 965, 525, ..., 0, 0, 0],
 ...,
 [ 1, 63, 20, ..., 0, 0, 0],
 [ 1, 63, 20, ..., 0, 0, 0],
 [ 1, 63, 20, ..., 0, 0, 0]], dtype=int32)

In [18]:
# Join the encoded titles to the dataset
new_dataset=new_dataset.join(pd.DataFrame(padded_titles))
new_dataset.head(10)

Out[7]: 
 book_id user_id rating original_title 0 1 ... 4 5 6 7 8 9
0 1 314 5 The Hunger Games 1 965 ... 0 0 0 0 0 0
1 1 439 3 The Hunger Games 1 965 ... 0 0 0 0 0 0
2 1 588 5 The Hunger Games 1 965 ... 0 0 0 0 0 0
3 1 1169 4 The Hunger Games 1 965 ... 0 0 0 0 0 0
4 1 1185 4 The Hunger Games 1 965 ... 0 0 0 0 0 0
5 1 2077 4 The Hunger Games 1 965 ... 0 0 0 0 0 0
6 1 2487 4 The Hunger Games 1 965 ... 0 0 0 0 0 0
7 1 2900 5 The Hunger Games 1 965 ... 0 0 0 0 0 0
8 1 3662 4 The Hunger Games 1 965 ... 0 0 0 0 0 0
9 1 3922 5 The Hunger Games 1 965 ... 0 0 0 0 0 0

[10 rows x 14 columns]

In [19]:
#split for training
train, test = train_test_split(new_dataset, test_size=0.1, random_state=42)


##Distributed training with Horovod

In [21]:
import time
FUSE_MOUNT_LOCATION = '/dbfs/horovod_keras/'
checkpoint_dir = FUSE_MOUNT_LOCATION + '{}/'.format(time.time())
os.makedirs(checkpoint_dir)

###Build a new hybrid recommender model
Use embedded item metadata in addition to user, item and rating information

In [23]:
def shard_data (train_data, rank=0, size=1):
  train_data=train_data[rank::size]
  return (train_data)

In [24]:
def get_model(n_items, n_users, embedding_size=5):
  item_input = Input(shape=[1], name="Item-Input")
  item_embedding = Embedding(input_dim=n_items+1, output_dim=embedding_size, name="Item-Embedding")(item_input)
  item_vec = Flatten(name="Flatten-Items")(item_embedding)

  user_input = Input(shape=[1], name="User-Input")
  user_embedding = Embedding(input_dim=n_users+1, output_dim=embedding_size, name="User-Embedding")(user_input)
  user_vec = Flatten(name="Flatten-Users")(user_embedding)

  metadata_input=Input(shape=[max_length], name="Metadata-Input")
  metadata_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_size, name="Metadata-Embedding")(metadata_input)
  metadata_vec = Flatten(name="Flatten-Metadata")(metadata_embedding)

  # new model with metadata and several regularization techniques to prevent overfitting
  input_vecs = Concatenate()([user_vec, item_vec, metadata_vec])
  input_vecs = Dropout(0.5)(input_vecs)
  x = Dense(128, activation='relu',
            #kernel_regularizer=regularizers.l2(0.01),
            kernel_initializer='random_normal',
            activity_regularizer=regularizers.l1(10e-4) #to prevent overfitting of training data
           )(input_vecs)
  #x = Dropout(0.5)(x)
  #output layer
  y = Dense(1)(x)

  new_model = Model([user_input, item_input, metadata_input], y)
  return (new_model)

In [25]:
## train new model with sampling
nb_epoch = 12
batch_size = 128 # batch is bigger because execution is split across GPUs
outputFilePath = "/dbfs/mnt/{}/{}".format(container, 'hybrid_deep_model.h5')

In [26]:
import keras
import horovod.keras as hvd

from keras import backend as K
import tensorflow as tf

def train_model(train, n_items, n_users):
  # Horovod: initialize Horovod.
  hvd.init()
  
  # Horovod: pin GPU to be used to process local rank (one GPU per process)
  config = tf.ConfigProto()
  config.gpu_options.allow_growth = True
  config.gpu_options.visible_device_list = str(hvd.local_rank())
  K.set_session(tf.Session(config=config))
  
  #get data
  train_data=shard_data(train, hvd.rank(), hvd.size())
  #get model
  new_model=get_model(n_items, n_users, 5)
  
  # Horovod: adjust learning rate based on number of GPUs.
  learning_rate=1.0
  optimizer = keras.optimizers.Adadelta(learning_rate * hvd.size())

  # Horovod: Wrap optimizer with Horovod DistributedOptimizer.
  optimizer = hvd.DistributedOptimizer(optimizer)
  
  #compile model
  new_model.compile(optimizer=optimizer, loss='mean_squared_error')
  
  # Horovod: Broadcast initial variable states from rank 0
  # to all other processes. This is necessary to ensure 
  # consistent initialization of all workers when training is
  # started with random weights or restored from a checkpoint.
  callbacks = [hvd.callbacks.BroadcastGlobalVariablesCallback(0)]
  
  # Horovod: Save checkpoints only on worker 0 to prevent 
  # other workers from overwriting and corrupting them.
  if hvd.rank() == 0:
    callbacks.append(keras.callbacks.ModelCheckpoint(filepath=outputFilePath, save_best_only=True))

  #more callbacks
  earlystop = EarlyStopping(monitor='val_loss',
                           patience=5,
                           verbose=1,
                           restore_best_weights=True
                           )
  callbacks.append(earlystop)
  # train model
  new_model.fit([train_data.user_id, train_data.book_id, train_data.iloc[:,4:4+max_length]], train_data.rating,
                      epochs=nb_epoch,
                      batch_size=batch_size,
                      shuffle=True,
                      validation_split=0.1,
                      verbose=2,
                      callbacks=callbacks)

In [27]:
from sparkdl import HorovodRunner

hr = HorovodRunner(np=2)
hr.run(train_model, train=train, n_items=n_items, n_users=n_users)

The global names read or written to by the pickled function are {'EarlyStopping', 'max_length', 'str', 'outputFilePath', 'keras', 'batch_size', 'hvd', 'shard_data', 'tf', 'K', 'get_model', 'nb_epoch'}.
The pickled object size is 68410695 bytes.
The pickled object size is greater than 10MB. It might cause training slow to start. You might
consider:
* Loading large datasets inside the main function instead of materializing them on the driver.
* Avoid pulling unnecessary variables from the notebook context.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Start training.

### Where to find training logs? ###
You can find the stdout and stderr of this training session in Spark UI. Expand the job progress bar
until you see the (i) icon next to the single stage from this job. Click it and then click "stderr"
corresponding to Task 0. You can use cluster log delivery feature to archive the logs.

In [28]:
new_model= load_model(outputFilePath)

In [29]:
eval_loss=new_model.evaluate([test.user_id, test.book_id, test.iloc[:, 4:4+max_length]], test.rating)
print("Evaluation loss: ", eval_loss)

32/98176 [..............................] - ETA: 4:09
 1120/98176 [..............................] - ETA: 11s 
 2304/98176 [..............................] - ETA: 7s 
 3776/98176 [>.............................] - ETA: 5s
 4768/98176 [>.............................] - ETA: 5s
 6272/98176 [>.............................] - ETA: 4s
 7520/98176 [=>............................] - ETA: 4s
 8928/98176 [=>............................] - ETA: 4s
10304/98176 [==>...........................] - ETA: 4s
11744/98176 [==>...........................] - ETA: 3s
12736/98176 [==>...........................] - ETA: 3s
13920/98176 [===>..........................] - ETA: 3s
15552/98176 [===>..........................] - ETA: 3s
16448/98176 [====>.........................] - ETA: 3s
16896/98176 [====>.........................] - ETA: 3s
17696/98176 [====>.........................] - ETA: 3s
18496/98176 [====>.........................] - ETA: 3s
19936/98176 [=====>........................] - ETA: 3s
21568/98176 [=====>........................] - ETA: 3s
23360/98176 [======>.......................] - ETA: 3s
25216/98176 [======>.......................] - ETA: 3s
27040/98176 [=======>......................] - ETA: 3s
28864/98176 [=======>......................] - ETA: 2s
30688/98176 [========>.....................] - ETA: 2s
32544/98176 [========>.....................] - ETA: 2s
34336/98176 [=========>....................] - ETA: 2s
36160/98176 [==========>...................] - ETA: 2s
38016/98176 [==========>...................] - ETA: 2s
39776/98176 [===========>..................] - ETA: 2s
41440/98176 [===========>..................] - ETA: 2s
43264/98176 [============>.................] - ETA: 2s
45120/98176 [============>.................] - ETA: 1s
46976/98176 [=============>................] - ETA: 1s
48768/98176 [=============>................] - ETA: 1s
50592/98176 [==============>...............] - ETA: 1s
51904/98176 [==============>...............] - ETA: 1s
52992/98176 [===============>..............] - ETA: 1s
54560/98176 [===============>..............] - ETA: 1s
56320/98176 [================>.............] - ETA: 1s
58176/98176 [================>.............] - ETA: 1s
60032/98176 [=================>............] - ETA: 1s
61888/98176 [=================>............] - ETA: 1s
63520/98176 [==================>...........] - ETA: 1s
65344/98176 [==================>...........] - ETA: 1s
67232/98176 [===================>..........] - ETA: 1s
68992/98176 [====================>.........] - ETA: 0s
70752/98176 [====================>.........] - ETA: 0s
72448/98176 [=====================>........] - ETA: 0s
74240/98176 [=====================>........] - ETA: 0s
76032/98176 [======================>.......] - ETA: 0s
77888/98176 [======================>.......] - ETA: 0s
79744/98176 [=======================>......] - ETA: 0s
81568/98176 [=======================>......] - ETA: 0s
83392/98176 [========================>.....] - ETA: 0s
85248/98176 [=========================>....] - ETA: 0s
87008/98176 [=========================>....] - ETA: 0s
88864/98176 [==========================>...] - ETA: 0s
90560/98176 [==========================>...] - ETA: 0s
92256/98176 [===========================>..] - ETA: 0s
94048/98176 [===========================>..] - ETA: 0s
95872/98176 [============================>.] - ETA: 0s
97600/98176 [============================>.] - ETA: 0s
98176/98176 [==============================] - 3s 32us/step
Evaluation loss: 0.7187896667842448

Less overfit, better mse overall!